## 開倉

In [4]:
exp_lose_money = 50
exp_lose_ratio = 0.5
maintain_ratio = 0.1

leveage = maintain_ratio / exp_lose_ratio
guarantee_money = exp_lose_money * leveage

print("leveage: ", leveage)
print("guarantee_money: ", guarantee_money)

leveage:  0.2
guarantee_money:  10.0


In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import mplfinance as mpf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz

In [54]:
import pytz
from datetime import datetime

timezone = pytz.timezone('Asia/Taipei')

start_time = timezone.localize(datetime(2024, 8, 20)).astimezone(pytz.UTC)
end_time = timezone.localize(datetime(2024, 8, 21)).astimezone(pytz.UTC)

In [82]:
def getcryptoprice(symbol, start_time, end_time, interval, limit):
    
    base_url = 'https://fapi.binance.com'
    symbol = symbol
    limit = limit
    interval = interval
    
    # Convert datetime to timestamp in milliseconds
    def timestamp_to_int(dt):
        return int(datetime.timestamp(dt) * 1000)
    
    start_time = timestamp_to_int(start_time)
    end_time = timestamp_to_int(end_time)

    # Convert UTC timestamp to Asia/Taipei timezone
    def convert_to_taipei_time(utc_timestamp):
        return datetime.fromtimestamp(utc_timestamp / 1000, pytz.timezone('Asia/Taipei'))

    def get_historical_klines(df, start_time):
        endpoint = '/fapi/v1/klines'
        Interval = interval
        params = {
            'symbol': symbol,
            'interval': Interval,
            'startTime': start_time,
            #'endTime': start_time + limit * 60 * 1000 - 1,
            'limit': 400
        }
        url = base_url + endpoint
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            klines_df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            
            # Convert timestamp to Asia/Taipei timezone
            klines_df['timestamp'] = klines_df['timestamp'].apply(convert_to_taipei_time)
            
            df = pd.concat([df, klines_df], ignore_index=True)
            #print(f'Symbol: {symbol}, Fetched {len(klines_df)} klines from {klines_df["timestamp"].min()} to {klines_df["timestamp"].max()} (Asia/Taipei)')
        else:
            print(f'Error: {response.status_code}')

        return df
    
    df = pd.DataFrame()

    # Fetch data in loop
    while start_time < end_time:
        df = get_historical_klines(df, start_time)
        start_time += 5 * 60 * 1000
        
    return df

def getcryptoOI(symbol, start_time, end_time, interval, limit):
    
    base_url = 'https://fapi.binance.com'
    symbol = symbol
    limit = limit
    period = interval
    
    # Convert datetime to timestamp in milliseconds
    def timestamp_to_int(dt):
        return int(datetime.timestamp(dt) * 1000)
    
    start_time = timestamp_to_int(start_time)
    end_time = timestamp_to_int(end_time)

    # Convert UTC timestamp to Asia/Taipei timezone
    def convert_to_taipei_time(utc_timestamp):
        return datetime.fromtimestamp(utc_timestamp / 1000, pytz.timezone('Asia/Taipei'))

    # Function to get historical OI and append to DataFrame
    def get_historical_OI(df, start_time):
        endpoint = '/futures/data/openInterestHist'
        params = {
            'symbol': symbol,
            'period': period,
            'startTime': start_time,
            #'endTime': start_time + limit * 60 * 1000 - 1,
            'limit': 400
        }
        url = base_url + endpoint
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            OI_df = pd.DataFrame(data, columns=['timestamp', 'sumOpenInterest', 'sumOpenInterestValue'])
            OI_df['timestamp'] = OI_df['timestamp'].apply(convert_to_taipei_time)
            df = pd.concat([df, OI_df], ignore_index=True)
            #print(f'Symbol: {symbol}, Fetched {len(OI_df)} OI from {OI_df["timestamp"].min()} to {OI_df["timestamp"].max()}')
        else:
            print(f'Error: {response.status_code}')

        return df
    
    df = pd.DataFrame()

    # Fetch data in loop
    while start_time < end_time:
        df = get_historical_OI(df, start_time)
        start_time += 5 * 60 * 1000  # Move to next 15 minutes

    return df

def getcryptoOIlongshort(symbol, start_time, end_time, interval, limit):
    
    base_url = 'https://fapi.binance.com'
    symbol = symbol
    limit = limit
    period = interval
    
    # Convert datetime to timestamp in milliseconds
    def timestamp_to_int(dt):
        return int(datetime.timestamp(dt) * 1000)
    
    start_time = timestamp_to_int(start_time)
    end_time = timestamp_to_int(end_time)

    # Convert UTC timestamp to Asia/Taipei timezone
    def convert_to_taipei_time(utc_timestamp):
        return datetime.fromtimestamp(utc_timestamp / 1000, pytz.timezone('Asia/Taipei'))\
    
    # Function to get historical OI and append to DataFrame
    def get_historical_longshort(df, start_time):
        endpoint = '/futures/data/globalLongShortAccountRatio'
        params = {
            'symbol': symbol,
            'period': period,
            'startTime': start_time,
            #'endTime': start_time + limit * 60 * 1000 - 1,
            'limit': 400
        }
        url = base_url + endpoint
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            longshort_df = pd.DataFrame(data, columns=['timestamp', 'longShortRatio', 'longaccount', 'shortaccount'])
            longshort_df['timestamp'] = longshort_df['timestamp'].apply(convert_to_taipei_time)
            df = pd.concat([df, longshort_df], ignore_index=True)
            #print(f'Symbol: {symbol}, Fetched {len(longshort_df)} longshort from {longshort_df["timestamp"].min()} to {longshort_df["timestamp"].max()}')
        else:
            print(f'Error: {response.status_code}')

        return df
    
    df = pd.DataFrame()

    # Fetch data in loop
    while start_time < end_time:
        df = get_historical_longshort(df, start_time)
        start_time += 5 * 60 * 1000  # Move to next 15 minutes

    return df

def getsignal(symbol, start_time, end_time, interval, limit):
    df = getcryptoprice(symbol, start_time, end_time, interval, limit)
    df_OI = getcryptoOI(symbol, start_time, end_time, interval, limit)
    df_longshort = getcryptoOIlongshort(symbol, start_time, end_time, interval, limit)
    
    df = pd.merge(df, df_OI, on='timestamp', how='left')
    df = pd.merge(df, df_longshort, on='timestamp', how='left')
    
    return df

In [83]:
signal = getsignal('AAVEUSDT',start_time,end_time,'5m',1000)

MemoryError: Unable to allocate 39.1 GiB for an array with shape (5243439888,) and data type int64

## 空

In [1]:
order_price = 0.04087
earn_ratio = 0.03
lost_ratio = 0.01

take_profit = order_price * (1-earn_ratio)
stop_loss = order_price * (1+lost_ratio)
print("take_profit: ", take_profit)
print("stop_loss: ", stop_loss)

take_profit:  0.039643899999999996
stop_loss:  0.041278699999999995


## 多

In [7]:
order_price = 0.7045
earn_ratio = 0.03
lost_ratio = 0.04

take_profit = order_price * (1+earn_ratio)
stop_loss = order_price * (1-lost_ratio)
print("take_profit: ", take_profit)
print("stop_loss: ", stop_loss)

take_profit:  0.725635
stop_loss:  0.67632
